In [1]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import ast
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import string
from IPython.display import Markdown
import textdistance
from fonctions import *
tqdm.pandas()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Données

In [2]:
texte_1 = 'sane cum de culminis nostri fulgore, velut ex sole radii, sic omnium largiente conditore prodeant dignitates ut prime lucis integritas minorati luminis non sensiat detrimenta, nam tanto magis regale sceptrum extollitur tantoque curia regiminis sollicitudinibus relevatur, quanto tribunal ipsius digniores circumspicit contribules ; hac igitur consideratione ducti, quia regni Francie moderamur habenas, solii nostri decus tam veterum dignitatum ornatibus confoveri quam etiam novis ardemus honoribus ampliari, et licet ad quorumlibet nostrorum fidelium munifice prosequenda servicia liberalitas nostra quadam generalitate sit proclivis, illis tamen est quadam specialitate debitrix et eorum vota prosequi liberaliter affectamus, in quibus sanguinis connexitatem et vinculum agnoscimus et puram fidei constanciam invenimus, ut ipsi, fervore devocionis accensi, reddantur de bono in melius circa majestatem regiam et ejus obsequia promptiores, et ne percepta dudum obsequia que conscribi perhenniter in libro memorie meruerunt preminentia gradus aboleat vel incompensata relinquat.'

pierre_1a = 'De fulgore throni cesarei, velut ex sole radii, sic certe prodeunt dignitates, ut prime lucis integritas non sentiat minorati luminis detrimenta, tantoque magis imperiale sceptrum extollitur, tanto cura regiminis pluria sollicitudinibus relevatur, quanto tribunal ipsius digniores in circuitu circumspicit contribules regiones. Hac igitur consideratione commoti, qui celesti providentia romani imperii moderamur habenas, solii nostri decus tam veterum dignitatum ornatibus confovemus, quam novis honoribus ampliamur.'

pierre_1b = 'Licet ad quorumlibet nostrorum fidelium, munifice prosequenda servicia liberalitatis nostre manus quadam generalitate sit habilis, illis tamen quadam specialitate fit debitrix, in quibus sanguinis idemptitatem agnoscimus et fidei pure constanciam invenimus […].'

texte_2 = 'decet excellentiam regiam pene duriciem quam juris rigor dictat aliquando pietatis mitigare, cum per hoc subditorum succrescat devocio et honor regius occulis augeatur.'

pierre_2 = 'satis nobis cedit ad gloriam et nostris fidelibus ad profectum, si motus nostros, quos armat potentia, sic mansuetudine decoremus, ut pene duriciem quam iuris rigor aliquando dictat in subditis clementie nostre molliamus unguentis, cum crescat in subditis devocionis vigor, dum pena remittitur, et dum devotorum succrescit cumulus nostri honoris per consequens titulus augeatur.'

texte_3 = 'Decus est principum subditorum votis clementer annuere ac eorum preces misericorditer exaudire ; etsi ad prosequenda vota fidelium regia liberalitas regulariter sit habilis, illud tamen occurrit precipuum, ut hiis qui reipublice se exposcunt, prefata liberalitas specialitate quadam graciosior et liberior porrigatur, ut sic per gratiam diligantur in populis et per Dominum tueantur in terris.'

pierre_3a = 'Decus est principum fidelium votis clementer annuere, ac supplicum preces misericorditer exaudire, ut sic per gratiam diligantur in populis et per dominium tueantur in terris.'

pierre_3b = 'licet ad prosequenda munificentie sic vota fidelium liberalitatis regie dextera generali quadam irregularitate sic sit habilis, illis tamen graciosa porrigitur quadam specialitate liberior, quos ad obsequia grata continuos fructus sic et utiles semper claris indiciis experimur.'

texte_4 = 'Dignum esse credimus ac consonum racioni atque concordabile reputamus ut hiis quos in legitimis actibus deffectus natalium impedit legitimacionis honor per principem reperiatur et, si quando super hiis magestatis regie favor seu benivolentia fidelium suorum aut eorum, qui regie magestatis fideles esse desiderant et in eadem fidelitate manere et perseverare noscuntur, supplicacione requiritur, liberaliter largiatur.'

pierre_4 = 'Dignum esse decrevimus et consentaneum rationi, ut hii quos interdum in legittimis actibus defectus natalium impedit, legittimacionis honore per principem reparentur, et si quando super hiis imperialis favor fidelium suorum supplicatione requiritur, liberaliter largiatur […].'

texte_5 = 'dignum esse credimus et summo Regi placere, si lapsis interdum per culpe obvicium celeri remedio subvenimus, cum per hoc crescat vigor in subditis et nostri honoris cumulus augeatur.'

pierre_5a = 'pulcrum in subditis humanitatis opus exequimur, si lapsis interdum per lubrice culpe convicium, misericordie celeri remedio subvenimus […].'

pierre_5b = 'cum crescat in subditis vigor, dum pena remittitur, et dum devotorum succrescit cumulus nostri honoris per consequens titulus augeatur.'

texte_6 = 'dignum opus prosequi credimus, innata nobis regali clemencia suadente, si interdum per lubrice culpe lapsis convicium ad indulgendum se nostra manus habilitet, rigorem tamen nostram taliter temperando quod partes justicie totaliter non excludat ; et licet ad hoc libenter erga omnes clemencie regie dexteram extendamus, illam tamen circa illos libentius extendimus in quibus devocionis et pure fidelitatis constanciam inviolabiliter reperimus.'

pierre_6a = 'nobile opus sedentis in solio clemencia suadente prosequimur, si per lubrica culpe deliramenta cadentibus, ad sublevanda remedia, cito se nostra manus habilitat, et indulgemus obnoxiis, veniam post reatum, pulcrum vindicte genus exstimantes ignoscere, et sic materialis gladii refrenare licentiam, ut seviendi voluntate sopita, temperemus interdum misericordie condimento rigorem.'

pierre_6b = 'pulcrum in subditis humanitatis opus exequimur, si lapsis interdum per lubrice culpe convicium, misericordie celeri remedio subvenimus. cum summum vindicte genus credentes ignoscere nolumus semper culpas delere per gladium, in quibus rigorem possumus temperamento clementie mitigare.'

pierre_6c = 'Quam sit favorabilis magnificentia cesaris et quam sit affabilis misericordiam petentibus vultus eius ex presentis negocii serie potestis aperte convincere, in quo sic est operatus iusticiam, ut per misericordie semitas dirigat gressus suos, sic misericorditer est profectus, ut partes iusticie non excludat […].'

texte_7 = 'egregium humanitatis opus in subditis agere credimus, innata nobis regali clementia suadente, si interdum per lubrica culpe delinimenta collapsis et veniam petentibus post reatum, humane condicionis fragilitate pensata, lenimento misericordie subvenimus, cum fervor devocionis et fidei sinceritas in subditis augeatur qui a culpis per regalem clementiam abluuntur.'

pierre_7a = 'nobile opus sedentis in solio clementia suadente prosequimur, si per lubrica culpe deliramenta cadentibus, ad sublevanda remedia, cito se nostra manus habilitat et indulgemus obnoxiis veniam post reatum, pulcrum vindicte genus exstimantes ignoscere, et sic materialis gladii refrenare licentiam, ut seviendi voluntate sopita, temperemus interdum misericordie condimento rigorem.'

pierre_7b = 'um crescat in eis etiam erga nos vigor devocionis et fidei, si fragilitatis humane memores aliquid severitati detrahimus, et que punire possemus in gladio, lenimento misericordie mitigamus.'

pierre_7c = 'pulcrum in subditis humanitatis opus exequimur, si lapsis interdum per lubrice culpe convicium, misericordie celeri remedio subvenimus, cum summum vindicte genus credentes ignoscere, nolumus semper culpas delere per gladium in quibus rigorem possumus temperamento clemencie mitigare […].'

texte_8 = 'egregium humanitatis opus in subditis prosequi credimus, innata nobis regali clementia suadente, si interdum per lubrice culpe lapsis convicium, humane condicionis fragilitate pensata, misericorditer subvenimus ; non est rigor justicie taliter in omnibus observandus, quin interdum possit et debeat linimento misericordie temperari.'

pierre_8 = 'pulcrum in subditis humanitatis opus exequimur, si lapsis interdum per lubrice culpe convicium, misericordie celeri remedio subvenimus, cum summum vindicte genus credentes ignoscere, nolumus semper culpas delere per gladium in quibus rigorem possumus temperamento clemencie mitigare […].'

texte_9 = 'lla decet regiam dignitatem frui mansuetudine et prerogativa virtutis ut, attendens quod, si culpa non precederet, locum venia non haberet, benignitas regia, si suum subditum quadam forsan necessitate potius quam voluntate a fide debita contigerit deviasse et, postmodum errorem suum sapienter advertens, ad gratiam reconciliationis admitti supplicat puro corde, paratis brachiis ipsum recipiat, oblita preterita culpa sua.'

pierre_9 = 'Illa fruimur ex imperii dignitate mansuetudine sic, ac prerogativa virtutis, ut quia nisi culpa precederet locum venia non haberet, sed quacumque hora subditus ad fidem nostram conversus fuerit, patulis brachiis recipiamus eumdem, obliti preterite culpe sue.'

texte_10 = 'In culpis delinquentium puniendis, personarum et temporum qualitate pensata, temperamus interdum gratie condimento rigorem.'

pierre_10 = 'nobile opus sedentis in solio clemencia suadente prosequimur, si per lubrica culpe deliramenta cadentibus, ad sublevanda remedia cito se nostra manus habilitat, et indulgemus obnoxiis veniam post reatum, pulcrum vindicte genus exstimantes ignoscere, et sic materialis gladii refrenare licentiam, ut seviendi voluntate sopita, temperemus interdum misericordie condimento rigorem.'

texte_11 = 'Inter cetera que sollicitudinis nostre cura quotidiana meditatione revolvit, illud occurrit nostre considerationi precipuum ut venerandus ortodoxe [sic] fidei nostre cultus nobis traditus per os prophetarum, apostolorum et quattuor evangelistarum nostre fidei catholice fidelium testium et scribarum, quorum sonus in orbem exiit universum, nostris temporibus augeatur subditique nostri in unionis federe pacis transquillitate gaudeant et fruantur ; ex hiis quidem potest augeri culminis nostri felicitas et subditorum nostrorum status servari pacificus, si Regi regum grato primitus devotionis obsequio placeamus et status ecclesiasticus in sua regularitate populusque nostro commissus regimini sub unionis nexu in sua integritate serventur.'

pierre_11 = 'Inter cetera, que sollicitudinis nostre cura cotidiana diligenti meditatione revolvit, illud occurrit nostre considerationi precipuum, qualiter venerandus orthodoxe fidei nostre cultus debito a nobis et nostris fidelibus affectu et devota veneratione colatur, ut quo pre ceteris principibus divina dextera nostrum erigit altius solium, eo pro [sic] per officiose gratitudinis debitum, nostra et nostrorum ubique devotio sibi solvat munera gratiarum. Ex eo si quidem potest regnantium felicitas augeri potissime et subditorum status servari pacificus, si Regi regum grato devotionis exennio placeamus, nec ex alicuius scandali suprectione dampnabili cultus debet postponi, vel intermitti dominicus, quia quo amplius humana scandala quandoque lasciviunt, eo sepefactus sedator esset devotius exorandus.'

texte_12 = 'Licet ad quorumlibet vota munifice prosequenda liberalitas nostra quadam generalitate sit proclivis, illis tamen est quadam specialitate debitrix quos ad regie celsitudinis grata obsequia promptos et utiles claris judiciis experimur.'

pierre_12a = 'Licet ad quorumlibet nostrorum fidelium, munifice prosequenda servicia liberalitatis nostre manus quadam generalitate sit habilis, illis tamen quadam specialitate fit debitrix, in quibus sanguinis idemptitatem agnoscimus et fidei pure constanciam invenimus […].'

pierre_12b = 'Licet ad prosequenda munificentie [sic] vota fidelium liberalitatis regie dextera generalita quadam irregularitate [sic] sit habilis, illis tamen graciosa porrigitur, quadam specialitate liberior, quos ad obsequia grata continuos fructus [sic] et utiles claris semper indiciis experimur.'

texte_13 = 'Opus insigne presidentis in solio in nobis clementia suggerente proficimus, si per lubrica culpe delinimenta labentibus misericordie celeri levamine subvenimus.'

pierre_13a = 'Opus in nobis egregium, clemencia suggerente, perficimus, si per lubrica culpe deliramenta labentibus, misericordie celeris remediosubvenimus '

pierre_13b = 'nobile opus sedentis in solio clemencia suadente prosequimur, si per lubrica culpe deliramenta cadentibus'

texte_14 = 'Pulcrum vindicte genus estimantes ignoscere, temperamus interdum gratie condimento rigorem.'

pierre_14 = 'nobile opus sedentis in solio clemencia suadente prosequimur, si per lubrica culpe deliramenta cadentibus, ad sublevanda remedia, se cito nostra manus habilitat, et indulgemus obnoxiis veniam post reatum, pulcrum vindicte genus exstimantes, ignoscere, et sic materialis gladii refrenare licentiam, ut seviendi voluntate sopita, temperemus interdum misericordie condimento rigorem.'

texte_15 = 'Satis nobis, quem Rex regum ad regie dignitatis fastigia provexit, cedit ad gloriam, si motus nostros virtute clementie perficimus et mansuetudinem in subditis excercemus, cum crescat etiam erga nos vigor devotionis et fidei, si, fragilitatis humane memores, aliquid severitati detrahimus et que punire possemus lenimento misericordie mittigamus.'

pierre_15 = 'Satis nobis cedit ad gloriam et regie dignitatis fastigia sublimamus sic, si motus nostros clemencie virtute perspicimus sic, et mansuetudinem in subditis exercemus, cum crescat in eis etiam erga nos vigor devocionis et fidei si fragilitatis humane memores aliquid severitati detrahimus, et que punire possemus in gladio, lenimento misericordie mitigamus'

# Fonctions

In [3]:
def remove_characters(input_string, characters_to_remove):
    translator = str.maketrans('', '', characters_to_remove)
    cleaned_string = input_string.translate(translator)
    return cleaned_string
characters_to_remove = string.punctuation + '[]' +'–'
def string_to_quadrigrams(input_string):
    quadrigram = [input_string[i:i+4] for i in range(len(input_string) - 3)]
    return quadrigram
def string_to_trigrams(input_string):
    trigram = [input_string[i:i+3] for i in range(len(input_string) - 2)]
    return trigram
def test_function_2(test_1,test_2):
    
    if test_1[0] == test_2[0] and test_1[1] == test_2[1] and test_1[2] == test_2[2]:
        return True
    
    elif test_1[0] == test_2[1] and test_1[1] == test_2[2] and test_1[2] == test_2[3]:
        return True

    elif test_1[1] == test_2[1] and test_1[2] == test_2[2] and test_1[3] == test_2[3]:
        return True
    
    elif test_1[1] == test_2[0] and test_1[2] == test_2[1] and test_1[3] == test_2[2]:
        return True
    
    elif test_1[0] == test_2[0] and test_1[1] == test_2[1] and test_1[3] == test_2[3]:
        return True
    
    elif test_1[0] == test_2[1] and test_1[1] == test_2[2] and test_1[3] == test_2[3]:
        return True

    elif test_1[0] == test_2[0] and test_1[1] == test_2[1] and test_1[3] == test_2[2]:
        return True
    
    elif test_1[0] == test_2[0] and test_1[2] == test_2[2] and test_1[3] == test_2[3]:
        return True
    
    elif test_1[0] == test_2[0] and test_1[2] == test_2[1] and test_1[3] == test_2[2]:
        return True
    
    elif test_1[0] == test_2[1] and test_1[2] == test_2[2] and test_1[3] == test_2[3]:
        return True
    
    
    
    elif test_1[0] == test_2[0] and test_1[1] == test_2[2] and test_1[2] == test_2[3]:
        return True    

    elif test_1[1] == test_2[0] and test_1[2] == test_2[2] and test_1[3] == test_2[3]:
        return True
    
    
    elif test_1[1] == test_2[0] and test_1[2] == test_2[1] and test_1[3] == test_2[3]:
        return True
    
    
    elif test_1[0] == test_2[0] and test_1[2] == test_2[1] and test_1[3] == test_2[3]:
        return True
    
    
    elif test_1[0] == test_2[0] and test_1[1] == test_2[2] and test_1[3] == test_2[3]:
        return True
    
    
    elif test_1[0] == test_2[1] and test_1[2] == test_2[2] and test_1[3] == test_2[3]:
        return True
    
    elif test_1[0] == test_2[0] and test_1[1] == test_2[1] and test_1[2] == test_2[3]:
        return True
    
    else:
        return False
    
def diff_text(quadri_1,quadri_2):
    different_texte = []
    for i in range(len(quadri_1)):
        if quadri_1[i] not in quadri_2:
            different_texte.append(i)
    return different_texte

def diff_pierre(quadri_1,quadri_2):
    different_pierre = []
    for i in range(len(quadri_2)):
        if quadri_2[i] not in quadri_1:
            different_pierre.append(i)
    return different_pierre

def delete_value(liste,index):
    han = []
    for i in range(len(liste)):
        if i != index:
            han.append(liste[i])
    return han
def convert_list_to_int(lst):
    return lst[0] if len(lst) > 0 else None

def find_common_words_and_transform(list1, index1, list2, index2):
    words_around_index1 = set(list1[max(0, index1-3):index1+4])
    words_around_index2 = set(list2[max(0, index2-3):index2+4])
    
    common_words = set()
    for word in words_around_index1 & words_around_index2:
        if len(word) >= 3:
            common_words.add(word)
    
    if common_words:
        for i in range(max(0, index1-3), min(len(list1), index1+4)):
            if list1[i] in common_words:
                list1[i] = f'**{list1[i]}**'
                list1[index1] = f'**{list1[index1]}**'
        for i in range(max(0, index2-3), min(len(list2), index2+4)):
            if list2[i] in common_words:
                list2[i] = f'**{list2[i]}**'
                list2[index2] = f'**{list2[index2]}**'

def bold_texts(texte,pierre):

    texte = remove_characters(texte,characters_to_remove).lower().lstrip().rstrip().replace('j','i').replace('v','u')
    pierre = remove_characters(pierre,characters_to_remove).lower().lstrip().rstrip().replace('j','i').replace('v','u')


    pierre_split = pierre.split()
    texte_split = texte.split()

    test_1 = string_to_quadrigrams(texte_split)
    test_2 = string_to_quadrigrams(pierre_split)
    

    #textdistance

    pierre_indice = []
    texte_indice= []

    pierre_mot = []
    texte_mot = []

    for idx1, word1 in enumerate(texte_split):
        for idx2, word2 in enumerate(pierre_split):
            similarity = textdistance.levenshtein.normalized_similarity(word1, word2)
            if 0.8 <= similarity < 1 and word1 in texte_split and word2 in pierre_split:
                pierre_indice.append(idx2)
                texte_indice.append(idx1)
                pierre_mot.append(word2)
                texte_mot.append(word1)

    for i in range(len(texte_indice)):

        index1 = texte_indice[i]  # Index pour la première liste
        index2 = pierre_indice[i]  # Index pour la deuxième liste

        find_common_words_and_transform(texte_split, index1, pierre_split, index2)

    ah = []
    for index in range(len(pierre_indice)-1):
        if pierre_indice[index]+1 == pierre_indice[index+1]:
            ah.append(pierre_indice[index])
            ah.append(pierre_indice[index]+1)
    ah = list(set(ah))

    for index in ah:
        if 0 <= index < len(pierre_split):
            pierre_split[index] = f'**{pierre_split[index]}**'

    ceh = []
    for index in range(len(texte_indice)-1):
        if texte_indice[index]+1 == texte_indice[index+1]:
            ceh.append(texte_indice[index])
            ceh.append(texte_indice[index]+1)
    ceh = list(set(ceh))

    for index in ceh:
        if 0 <= index < len(texte_split):
            texte_split[index] = f'**{texte_split[index]}**'

    i = 0
    indices_textes = []
    while i < len(test_1):
        indices_textes.append([i,i+1,i+2,i+3])
        i+=1

    i = 0    
    indices_pierre = []
    while i < len(test_2):
        indices_pierre.append([i,i+1,i+2,i+3])
        i+=1

    liste_quadri_textes = []
    for item in test_1:
        liste_quadri_textes.extend([item]*len(test_2))
    liste_quadri_pierre = test_2 * len(test_1)

    ind_textes = []

    for item in indices_textes:
        ind_textes.extend([item]*len(indices_pierre))

    ind_pierre = indices_pierre * len(indices_textes)

    df = pd.DataFrame({'quadri_textes':liste_quadri_textes,'quadri_pierre':liste_quadri_pierre})
    df['common'] = df.apply(lambda row: test_function_2(row['quadri_textes'],row['quadri_pierre']), axis=1)
    df['indices_textes'] = ind_textes
    df['indices_pierre'] = ind_pierre
    df = df[df['common'] == True].reset_index(drop=True)
    
    if df.empty:
        return False
    else:
    
        df = df[['quadri_textes','quadri_pierre','indices_textes','indices_pierre']]
        df['diff_texte'] = df.apply(lambda row: diff_text(row['quadri_textes'],row['quadri_pierre']), axis=1)
        df['diff_pierre'] = df.apply(lambda row: diff_pierre(row['quadri_textes'],row['quadri_pierre']), axis=1)

        df = df[df['diff_texte'].apply(lambda x: len(x) > 0)]
        df = df[df['diff_pierre'].apply(lambda x: len(x) > 0)]
        df['diff_texte'] = df['diff_texte'].apply(convert_list_to_int)
        df['diff_pierre'] = df['diff_pierre'].apply(convert_list_to_int)

        df['texte_ind'] = df.apply(lambda row:delete_value(row['indices_textes'],row['diff_texte']), axis=1)
        df['pierre_ind'] = df.apply(lambda row:delete_value(row['indices_pierre'],row['diff_pierre']), axis=1)

        flattened_textes = [item for sublist in df['texte_ind'].tolist() for item in sublist]
        flattened_pierre = [item for sublist in df['pierre_ind'].tolist() for item in sublist]

        liste_idx_texte = list(set(flattened_textes))
        liste_idx_pierre = list(set(flattened_pierre))

        for index in liste_idx_texte:
            if 0 <= index < len(texte_split):
                texte_split[index] = f'**{texte_split[index]}**'
        for index in liste_idx_pierre:
            if 0 <= index < len(pierre_split):
                pierre_split[index] = f'**{pierre_split[index]}**'

    
        return texte_split,pierre_split

    
def process_string(texte):
    processed_text = texte.lower().lstrip().rstrip().replace('j', 'i').replace('v', 'u')
    return processed_text

def from_tuples_to_column(tuples):
    col1,col2 = tuples
    return col1,col2

def indice_etoiles(liste):
    stars = []

    for i in range(len(liste)):
        if liste[i].startswith('**'):
            stars.append(i)
    return stars

def length(liste):
    return len(liste)

def to_bold(liste,liste_index):
    
    for i in liste_index:
        liste[i] = f'**{liste[i]}**'
    return liste
def find_que(liste):
    return any(word.endswith('que') for word in liste)
def index_que(liste):
    index = []
    for i in range(len(liste)):
        if liste[i].endswith('que') and len(liste[i]) > 3:
            index.append(i)
    return index

def lengths(liste1,liste2):
    return len(liste1) == len(liste2)

def transform_word(liste):
    new_list = []
    for word in liste:
        if word.startswith('**'):
            cleaned_word = word.replace('*', '')

            new_word = '**' + cleaned_word + '**'
            
            new_list.append(new_word)
        else:
            new_list.append(word)
    return new_list

def segment_lists(liste, start_index, end_index, context_size=10):
    start = max(start_index - context_size, 0)
    end = min(end_index + context_size + 1, len(liste))
    
    liste_finale = liste[start:end]
    return liste_finale

def que(liste):
    for word in range(len(liste)):
        if liste[word].endswith('que') and len(liste[word]) > 3:
            return True
    return False   

def check_len(liste1,liste2):
    return len(liste1) == len(liste2)
def jarod(liste1,liste2): 
    if que(liste1) == True and check_len(liste1,liste2) == False and len(index_que(liste1)) > 0:
        
        index = index_que(liste1)
        for i in range(len(index)):
            index[i] = index[i] +i

        mots_trans = []

        for i in index:
            if liste2[i+1] == 'que':
                mots_trans.append(liste2[i]+'que')

        for i in range(len(mots_trans) - 1, -1, -1):
            liste2[index[i]] = mots_trans[i]

        for i in range(len(index)):
            index[i] = index[i] +1

        for index in sorted(index, reverse=True):
            if 0 <= index < len(liste2):
                liste2.pop(index)

        return liste2
    else:
        return liste2
    
    
def lemmas_bold(x,y):
    
    match = []
    liste_pos = ['ADJqua','VER','NOMcom','ADV']

    for i in range(len(x)):
        if not x[i].startswith('**') and x[i] in y:
            match.append(x[i])
    final = []

    for i in match:
        if new_dic.get(i) in liste_pos:
            final.append(i)

    for i in range(len(x)):
        if x[i] in final:
            x[i] = f'**{x[i]}**'

    for i in range(len(y)):
        if y[i] in final:
            y[i] = f'**{y[i]}**'
    
    return x,y

def join(liste):
    return ' '.join(liste)
def cosine_sim(x,y):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([x, y])
    cosine_sim = cosine_similarity(tfidf_matrix)
    return cosine_sim[0][1]

def avoir_trigram_commun(phrase1, phrase2):
    trigrams_phrase1 = set(tuple(phrase1.split()[i:i+3]) for i in range(len(phrase1.split()) - 2))
    trigrams_phrase2 = set(tuple(phrase2.split()[i:i+3]) for i in range(len(phrase2.split()) - 2))
    
    return bool(trigrams_phrase1 & trigrams_phrase2)

def string_to_trigrams(input_string):
    trigrams = [input_string[i:i+3] for i in range(len(input_string) - 2)]
    return trigrams
def remove_lists(row):
    return [sublist for sublist in row if sublist not in trigrams_delete]
def empty_lists(liste):
    return liste == []
def length(liste):
    return len(liste)
def same(liste):
    if not liste:
        return False
    first_element = liste[0]
    for sublist in liste[1:]:
        if sublist != first_element:
            return False
    return True

def check_values(list1, list2):

    counter = 0

    for pos in list1:
        if pos in list2:
            counter += 1
    if counter > 1:
        return True
    else:
        return False

def pos_list(liste):
    y = []
    for i in liste:
        z = []
        for word in i:
            z.append(dic3.get(word))
        y.append(z)
        
    lol = []
    for i in range(len(y)):
        if check_values(y[i],liste_pos) == True:
            lol.append(liste[i])
    return lol

def copies(liste1,liste2):
    nouvelles_copies = []

    for element in liste1:
        nouvelles_copies.extend([element] * len(liste2))

    return nouvelles_copies


def avoir_quadrigramme_commun(texte,pierre):

    texte = remove_characters(texte,characters_to_remove).lower().lstrip().rstrip().replace('j','i').replace('v','u')
    pierre = remove_characters(pierre,characters_to_remove).lower().lstrip().rstrip().replace('j','i').replace('v','u')


    pierre_split = pierre.split()
    texte_split = texte.split()

    test_1 = string_to_quadrigrams(texte_split)
    test_2 = string_to_quadrigrams(pierre_split)
    

    #textdistance

    pierre_indice = []
    texte_indice= []

    pierre_mot = []
    texte_mot = []

    for idx1, word1 in enumerate(texte_split):
        for idx2, word2 in enumerate(pierre_split):
            similarity = textdistance.levenshtein.normalized_similarity(word1, word2)
            if 0.8 <= similarity < 1 and word1 in texte_split and word2 in pierre_split:
                pierre_indice.append(idx2)
                texte_indice.append(idx1)
                pierre_mot.append(word2)
                texte_mot.append(word1)

    for i in range(len(texte_indice)):

        index1 = texte_indice[i]  # Index pour la première liste
        index2 = pierre_indice[i]  # Index pour la deuxième liste

        find_common_words_and_transform(texte_split, index1, pierre_split, index2)

    ah = []
    for index in range(len(pierre_indice)-1):
        if pierre_indice[index]+1 == pierre_indice[index+1]:
            ah.append(pierre_indice[index])
            ah.append(pierre_indice[index]+1)
    ah = list(set(ah))

    for index in ah:
        if 0 <= index < len(pierre_split):
            pierre_split[index] = f'**{pierre_split[index]}**'

    ceh = []
    for index in range(len(texte_indice)-1):
        if texte_indice[index]+1 == texte_indice[index+1]:
            ceh.append(texte_indice[index])
            ceh.append(texte_indice[index]+1)
    ceh = list(set(ceh))

    for index in ceh:
        if 0 <= index < len(texte_split):
            texte_split[index] = f'**{texte_split[index]}**'

    i = 0
    indices_textes = []
    while i < len(test_1):
        indices_textes.append([i,i+1,i+2,i+3])
        i+=1

    i = 0    
    indices_pierre = []
    while i < len(test_2):
        indices_pierre.append([i,i+1,i+2,i+3])
        i+=1

    liste_quadri_textes = []
    for item in test_1:
        liste_quadri_textes.extend([item]*len(test_2))
    liste_quadri_pierre = test_2 * len(test_1)

    ind_textes = []

    for item in indices_textes:
        ind_textes.extend([item]*len(indices_pierre))

    ind_pierre = indices_pierre * len(indices_textes)

    df = pd.DataFrame({'quadri_textes':liste_quadri_textes,'quadri_pierre':liste_quadri_pierre})
    df['common'] = df.apply(lambda row: test_function_2(row['quadri_textes'],row['quadri_pierre']), axis=1)
    df['indices_textes'] = ind_textes
    df['indices_pierre'] = ind_pierre
    df = df[df['common'] == True].reset_index(drop=True)
    
    if df.empty:
        return False
    else:
        return True
    
def split(liste):
    return liste.split()

def compare_quadrigram_lists(list1, list2):
    for quadrigram1 in list1:
        for quadrigram2 in list2:
            intersection_count = len(set(quadrigram1) & set(quadrigram2))
            if intersection_count >= 3:
                return True
    return False

# Application de l'algorithme

In [4]:
delete_quadrigramme = [['notum','facimus','uniuersis','tam'],['tam', 'presentibus', 'quam', 'futuris'],
                 ['uniuersis', 'tam', 'presentibus', 'quam'],['rex', 'notum', 'facimus', 'uniuersis'],
                 ['facimus', 'uniuersis', 'tam', 'presentibus'],['presentibus', 'quam', 'futuris', 'nos'],
                 ['presentibus', 'quam', 'futuris', 'quod'],['etc', 'notum', 'facimus', 'uniuersis'],
                 ['et', 'ex', 'certa', 'scientia'],['notum', 'facimus', 'uniuersis', 'presentibus'],
                 ['quam', 'futuris', 'quod', 'cum'],['quam', 'futuris', 'quod', 'nos'],
                 ['de', 'speciali', 'gratia', 'et'],['in', 'regno', 'nostro', 'usque'],
                 ['gratia', 'et', 'ex', 'certa'],['in', 'regno', 'nostro', 'predicto'],
                 ['presentibus', 'et', 'futuris', 'quod'],['notum', 'facimus', 'quod', 'cum'],
                 ['quam', 'futuris', 'nos', 'in'],['de', 'regno', 'nostro', 'in'],
                 ['et', 'ex', 'certa', 'sciencia'],['et', 'in', 'regno', 'nostro'],
                 ['in', 'regno', 'nostro', 'et'],['facimus', 'tam', 'presentibus', 'quam']]

In [5]:
liste_texte = [texte_1,texte_2,texte_3,texte_4,texte_5,texte_6,texte_7,texte_8,texte_9,texte_10,texte_11,
              texte_12,texte_13,texte_14,texte_15]
liste_pierre = [pierre_1a,pierre_1b,pierre_2,pierre_3a,pierre_3b,pierre_4,pierre_5a,pierre_5b,pierre_6a,
                pierre_6b,pierre_6c,pierre_7a,pierre_7b,pierre_7c,pierre_8,pierre_9,pierre_10,pierre_11,
               pierre_12a,pierre_12b,pierre_13a,pierre_13b,pierre_14,pierre_15]

indice_pierre = ['pierre_1a', 'pierre_1b', 'pierre_2', 'pierre_3a', 'pierre_3b', 'pierre_4',
                'pierre_5a', 'pierre_5b', 'pierre_6a', 'pierre_6b', 'pierre_6c', 'pierre_7a',
                'pierre_7b', 'pierre_7c', 'pierre_8', 'pierre_9', 'pierre_10', 'pierre_11',
                'pierre_12a', 'pierre_12b', 'pierre_13a', 'pierre_13b', 'pierre_14', 'pierre_15']

indice_texte = ['texte_1', 'texte_2', 'texte_3', 'texte_4', 'texte_5', 'texte_6', 'texte_7', 'texte_8', 
               'texte_9', 'texte_10', 'texte_11', 'texte_12', 'texte_13', 'texte_14', 'texte_15']
trigrams_delete = [['tam', 'presentibus', 'quam'],['notum', 'facimus', 'uniuersis'],
                  ['presentibus', 'quam', 'futuris'],['uniuersis', 'tam', 'presentibus'],
                  ['quam', 'futuris', 'quod'],['de', 'speciali', 'gratia'],
                  ['et', 'ex', 'certa'],['ex', 'certa', 'scientia'],['speciali', 'gratia', 'et'],
                  ['futuris', 'quod', 'nos'],['presentes', 'litteras', 'inspecturis'],
                   ['uniuersis', 'presentes', 'litteras'],
                  ['facimus','uniuersis','tam'],['de','regno','nostro'],
                  ['in','regno','nostro'],['et','regni','nostri'],['regni','nostri','et'],
                   ['presentibus', 'et', 'futuris'],['per','regnum','nostrum']]

In [6]:
liste_texte = [process_string(remove_characters(texte, characters_to_remove)) for texte in liste_texte]
liste_pierre = [process_string(remove_characters(texte, characters_to_remove)) for texte in liste_pierre]

In [7]:
text_list = []
for item in liste_texte:
    text_list.extend([item]*len(liste_pierre))
pierre_list = liste_pierre * len(liste_texte)

text_index = []
for item in indice_texte:
    text_index.extend([item]*len(indice_pierre))
pierre_index = indice_pierre * len(indice_texte)

In [8]:
data = pd.DataFrame({'textes':text_list,'indice_text':text_index,'pierre':pierre_list,'indice_pierre':pierre_index})

In [9]:
x = liste_texte

from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)


lemmes_textes = []
for i in tqdm(range(len(x))):
    sentences: List[str] = [x[i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
        lemmes_textes.append(dic)
        
y = liste_pierre

from typing import List
from pie_extended.cli.utils import get_tagger, get_model, download

# In case you need to download
do_download = False
if do_download:
    for dl in download("lasla"):
        x = 1

# model_path allows you to override the model loaded by another .tar
model_name = "lasla"
tagger = get_tagger(model_name, batch_size=256, device="cpu", model_path=None)


lemmes_pierre = []
for i in tqdm(range(len(y))):
    sentences: List[str] = [y[i]]
    # Get the main object from the model (: data iterator + postprocesor
    from pie_extended.models.lasla.imports import get_iterator_and_processor
    for sentence_group in sentences:
        iterator, processor = get_iterator_and_processor()
        dic = tagger.tag_str(sentence_group, iterator=iterator, processor=processor)
        lemmes_pierre.append(dic)

fatal: HEAD n'est pas un nom d'objet valide
100%|███████████████████████████████████████████| 24/24 [00:06<00:00,  3.55it/s]


In [10]:
#création du dictionnaire de pos

dic1 = [item for sublist in lemmes_textes for item in sublist]
dic2 = [item for sublist in lemmes_pierre for item in sublist]

dic1.extend(dic2)

mots = []
part_of_speech = []

for dic in dic1:
    mots.append(dic.get('form'))
    part_of_speech.append(dic.get('pos'))

dic3 = dict(zip(mots,part_of_speech))

liste_pos = ['ADJqua','VER','ADV','NOMcom']

maxime = []
for i in range(len(lemmes_textes)):
    alexandra = []
    for j in lemmes_textes[i]:
        if j.get('pos') in liste_pos:
            alexandra.append(j.get('lemma'))
    maxime.append(alexandra)

franck = []
for i in range(len(lemmes_textes)):
    clemi = []
    for j in lemmes_textes[i]:
        clemi.append(j.get('lemma'))
    franck.append(clemi)
    
thibault = []
for i in range(len(lemmes_pierre)):
    aline = []
    for j in lemmes_pierre[i]:
        if j.get('pos') in liste_pos:
            aline.append(j.get('lemma'))
    thibault.append(aline)
romain = []
for i in range(len(lemmes_pierre)):
    dorothee = []
    for j in lemmes_pierre[i]:
        dorothee.append(j.get('lemma'))
    romain.append(dorothee)

un = copies(maxime,thibault)
deux = thibault*len(maxime)
trois = copies(indice_texte,thibault)
quatre = indice_pierre*len(maxime)
cinq = copies(franck,thibault)
six = romain*len(maxime)

data['texte_lemmes_cos'] = un
data['texte_lemmes_cos'] = data['texte_lemmes_cos'].apply(join)

data['pierre_lemmes_cos'] = deux
data['pierre_lemmes_cos'] = data['pierre_lemmes_cos'].apply(join)

data['texte_lemmes'] = cinq
data['texte_lemmes'] = data['texte_lemmes'].apply(join)

data['pierre_lemmes'] = six
data['pierre_lemmes'] = data['pierre_lemmes'].apply(join)

data['cosine'] = data.progress_apply(lambda x:cosine_sim(x['texte_lemmes_cos'],x['pierre_lemmes_cos']),axis=1)

data['quadri_texte'] = data['textes'].apply(split).apply(string_to_quadrigrams)
data['quadri_pierre'] = data['pierre'].apply(split).apply(string_to_quadrigrams)

data['compare_quadri'] = data.progress_apply(lambda x: compare_quadrigram_lists(x['quadri_texte'],x['quadri_pierre']),axis=1)

anais = data[data['cosine'] > 0.05].reset_index(drop=True)
anais = anais[anais['compare_quadri'] == True].reset_index(drop=True)


100%|███████████████████████████████████████| 360/360 [00:00<00:00, 1553.92it/s]


In [11]:
def list_of_common_quadrigrams(list1, list2):
    
    quadri1 = []
    quadri2 = []
    
    for quadrigram1 in list1:
        for quadrigram2 in list2:
            intersection_count = len(set(quadrigram1) & set(quadrigram2))
            if intersection_count >= 3:
                quadri1.append(quadrigram1)
                quadri2.append(quadrigram2)
    if len(quadri1) == 0:
        return False
    else:
        return quadri1,quadri2
def keep_first_element(liste):
    return liste[0]
def delete_quadri(liste,liste_delete):
    final_liste = []
    for quadri in liste:
        if not quadri in liste_delete:
            final_liste.append(quadri)
    if len(final_liste) == 0:
        return False
    else:
        return final_liste

In [12]:
data = data[data['cosine'] > 0.05].reset_index(drop=True)
data['quadri_texte'] = data['textes'].progress_apply(split).progress_apply(string_to_quadrigrams)
data['quadri_pierre'] = data['pierre'].progress_apply(split).progress_apply(string_to_quadrigrams)
data['compare'] = data.progress_apply(lambda x: list_of_common_quadrigrams(x['quadri_texte'],x['quadri_pierre']),axis=1)

100%|███████████████████████████████████████| 147/147 [00:00<00:00, 1283.43it/s]


In [13]:
oscar = data[data['compare'] != False].reset_index(drop=True)

In [14]:
oscar = oscar[['textes','indice_text','pierre','indice_pierre','texte_lemmes','pierre_lemmes']]

In [15]:
oscar['quadri_textes'] = oscar['textes'].apply(split).apply(string_to_quadrigrams)
oscar['quadri_pierre'] = oscar['pierre'].apply(split).apply(string_to_quadrigrams)

In [16]:
x = oscar['quadri_textes'][0]
y = oscar['quadri_textes'][1]

In [17]:
def compare_quadrigram_lists(list1, list2):
    quadri = []
    for quadrigram1 in list1:
        for quadrigram2 in list2:
            intersection_count = len(set(quadrigram1) & set(quadrigram2))
            if intersection_count < 3:
                return False
            elif test_function_2(quadrigram1,quadrigram2) == True:
                quadri.append(quadrigram1)
    return quadri

In [18]:
compare_quadrigram_lists(x,y)

False

In [19]:
data['text']

KeyError: 'text'

In [20]:
data

,textes,indice_text,pierre,indice_pierre,texte_lemmes_cos,pierre_lemmes_cos,texte_lemmes,pierre_lemmes,cosine,quadri_texte,quadri_pierre,compare_quadri,compare
0,sane cum de culminis nostri fulgore uelut ex s...,texte_1,de fulgore throni cesarei uelut ex sole radii ...,pierre_1a,sane culmen fulgor sol radius sic largior cond...,fulgor thronus cesareus sol radius sic certe p...,sane cum3 de culmen noster fulgor uelut1 ex so...,de fulgor thronus cesareus uelut1 ex sol radiu...,0.357799,"[[sane, cum, de, culminis], [cum, de, culminis...","[[de, fulgore, throni, cesarei], [fulgore, thr...",True,"([[fulgore, uelut, ex, sole], [fulgore, uelut,..."
1,sane cum de culminis nostri fulgore uelut ex s...,texte_1,licet ad quorumlibet nostrorum fidelium munifi...,pierre_1b,sane culmen fulgor sol radius sic largior cond...,fidelis2 munifice prosequor seruicium liberali...,sane cum3 de culmen noster fulgor uelut1 ex so...,licet3 ad2 quisque2 noster fidelis2 munifice p...,0.253344,"[[sane, cum, de, culminis], [cum, de, culminis...","[[licet, ad, quorumlibet, nostrorum], [ad, quo...",True,"([[et, licet, ad, quorumlibet], [licet, ad, qu..."
2,sane cum de culminis nostri fulgore uelut ex s...,texte_1,decus est principum fidelium uotis clementer a...,pierre_3a,sane culmen fulgor sol radius sic largior cond...,decus sum1 princeps1 fidelis2 uotum clementer ...,sane cum3 de culmen noster fulgor uelut1 ex so...,decus sum1 princeps1 fidelis2 uotum clementer ...,0.077837,"[[sane, cum, de, culminis], [cum, de, culminis...","[[decus, est, principum, fidelium], [est, prin...",False,False
3,sane cum de culminis nostri fulgore uelut ex s...,texte_1,licet ad prosequenda munificentie sic uota fid...,pierre_3b,sane culmen fulgor sol radius sic largior cond...,prosequor munificentiens sic uotum fidelis2 li...,sane cum3 de culmen noster fulgor uelut1 ex so...,licet3 ad2 prosequor munificentiens sic uotum ...,0.145148,"[[sane, cum, de, culminis], [cum, de, culminis...","[[licet, ad, prosequenda, munificentie], [ad, ...",True,"([[sit, procliuis, illis, tamen]], [[sit, habi..."
4,sane cum de culminis nostri fulgore uelut ex s...,texte_1,dignum esse decreuimus et consentaneum rationi...,pierre_4,sane culmen fulgor sol radius sic largior cond...,dignus sum1 decerno consentaneus ratio interdu...,sane cum3 de culmen noster fulgor uelut1 ex so...,dignus sum1 decerno et2 consentaneus ratio ut4...,0.080151,"[[sane, cum, de, culminis], [cum, de, culminis...","[[dignum, esse, decreuimus, et], [esse, decreu...",False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,satis nobis quem rex regum ad regie dignitatis...,texte_15,pulcrum in subditis humanitatis opus exequimur...,pierre_8,satis1 rex rex regie dignitas fastigium proueh...,pulcher subdo humanitas opus1 exsequor labor2 ...,satis1 nos qui1 rex rex ad2 regie dignitas fas...,pulcher in subdo humanitas opus1 exsequor si2 ...,0.051568,"[[satis, nobis, quem, rex], [nobis, quem, rex,...","[[pulcrum, in, subditis, humanitatis], [in, su...",False,False
143,satis nobis quem rex regum ad regie dignitatis...,texte_15,illa fruimur ex imperii dignitate mansuetudine...,pierre_9,satis1 rex rex regie dignitas fastigium proueh...,fruor imperium dignitas mansuetudo sic preroga...,satis1 nos qui1 rex rex ad2 regie dignitas fas...,ille fruor ex imperium dignitas mansuetudo sic...,0.098724,"[[satis, nobis, quem, rex], [nobis, quem, rex,...","[[illa, fruimur, ex, imperii], [fruimur, ex, i...",False,False
144,satis nobis quem rex regum ad regie dignitatis...,texte_15,inter cetera que sollicitudinis nostre cura co...,pierre_11,satis1 rex rex regie dignitas fastigium proueh...,cetera1 sollicitudo cura cotidianus diligens m...,satis1 nos qui1 rex rex ad2 regie dignitas fas...,inter cetera1 que sollicitudo noster cura coti...,0.096606,"[[satis, nobis, quem, rex], [nobis, quem, rex,...","[[inter, cetera, que, sollicitudinis], [cetera...",False,False
145,satis nobis quem rex regum ad regie dignitatis...,texte_15,opus in nobis egregi

In [ ]:
data['quadri'] = data.progress_apply(lambda x: avoir_trigram_commun(x['textes'],x['pierre']),axis=1)

#test = data[data['quadri'] == True]
anais = data[data['cosine'] > 0.05].reset_index(drop=True)

han = []
trigrams = []

for i in tqdm(range(len(anais))):
    inter = []
    for tri in string_to_trigrams(anais['textes'][i].split()):
        if tri in string_to_trigrams(anais['pierre'][i].split()):
            inter.append(tri)
    han.append(len(inter))
    trigrams.append(inter)
anais['trigrams'] = trigrams
anais['len'] = han
anais['grams'] = anais['trigrams'].apply(remove_lists)

anais['trigrams'] = trigrams
anais['len'] = han
anais['grams'] = anais['trigrams'].apply(remove_lists)
anais['empty'] = anais['grams'].apply(empty_lists)
lucie = anais[anais['empty'] == False].reset_index(drop=True)
#lucie = lucie[['textes','indice_text','pierre','indice_pierre','texte_lemmes','pierre_lemmes','grams']]
#lucie['len'] = lucie['grams'].apply(length)
#michelle = lucie[lucie['len']>1]
#michelle['same'] = michelle['grams'].apply(same)
#emile = michelle[michelle['same'] == False].reset_index(drop=True)
#emile['pos'] = emile['grams'].apply(pos_list)
#emile['empty'] = emile['pos'].apply(empty_lists)


In [ ]:
vincent = emile[emile['empty'] == False].reset_index(drop=True)

In [ ]:
x,y = bold_texts(texte_10,pierre_10)

In [ ]:
final_texte = ' '.join(x)
pierre_final = ' '.join(y)

print('----------')
display(Markdown(final_texte))
print('----------')
display(Markdown(pierre_final))    
print('----------')

In [ ]:
def get_common_word_trigrams(str1, str2):
    def extract_word_trigrams(string):
        words = string.split()
        return {tuple(words[i:i+3]) for i in range(len(words) - 2)}
    
    trigrams1 = extract_word_trigrams(str1)
    trigrams2 = extract_word_trigrams(str2)
    
    common_trigrams = trigrams1 & trigrams2
    
    if common_trigrams:
        return [trigram for trigram in common_trigrams]
    else:
        return False

common_word_trigrams = get_common_word_trigrams(string1, string2)

if common_word_trigrams:
    print("Common Word Trigrams:")
    for trigram in common_word_trigrams:
        print(list(trigram))
else:
    print("No common word trigrams found.")
def remove_lists(liste):
    
    if liste is False:
        return None
    else:
        return [sublist for sublist in liste if sublist not in trigrams_delete]
trigrams_delete = [['tam', 'presentibus', 'quam'],['notum', 'facimus', 'uniuersis'],
                  ['presentibus', 'quam', 'futuris'],['uniuersis', 'tam', 'presentibus'],
                  ['quam', 'futuris', 'quod'],['de', 'speciali', 'gratia'],
                  ['et', 'ex', 'certa'],['ex', 'certa', 'scientia'],['speciali', 'gratia', 'et'],
                  ['futuris', 'quod', 'nos'],['presentes', 'litteras', 'inspecturis'],
                   ['uniuersis', 'presentes', 'litteras'],
                  ['facimus','uniuersis','tam'],['de','regno','nostro'],
                  ['in','regno','nostro'],['et','regni','nostri'],['regni','nostri','et'],
                   ['presentibus', 'et', 'futuris'],['per','regnum','nostrum']]